In [8]:
import cv2
import numpy as np
import sklearn
import csv
from keras import optimizers

In [11]:
def generator(samples, batch_size=256):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                center_image = cv2.imread(batch_sample[0])
                if(batch_sample[2] == "flipped"):
                    center_image = cv2.flip(center_image, 1)
                angle = float(batch_sample[1])
                images.append(center_image)
                angles.append(angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

def BringToCount(imageArray, count):
    newImageArray = []
    while len(newImageArray) < count*0.7:
        for image in imageArray:
            newImageArray.append(image.copy())
    return newImageArray

def GetEqualShareLeftRightStraight(imageArray):
    notStraightAngle = 0.07
    left = []
    straight = []
    right = []
    imagesDict = {"left": [], "straight":[], "right":[]}
    for i in range(len(imageArray)):
        if(imageArray[i][1] > -notStraightAngle and imageArray[i][1] < notStraightAngle):
            imagesDict["straight"].append(imageArray[i])
        elif(imageArray[i][1] < notStraightAngle):
            imagesDict["left"].append(imageArray[i])
        elif(imageArray[i][1] > notStraightAngle):
            imagesDict["right"].append(imageArray[i])
    print("left ", len(imagesDict["left"]))
    print("straight ", len(imagesDict["straight"]))
    print("right", len(imagesDict["right"]))
    imagesDict["left"] = BringToCount(imagesDict["left"], len(imagesDict["straight"]))
    imagesDict["right"] = BringToCount(imagesDict["right"], len(imagesDict["straight"]))
    print("left ", len(imagesDict["left"]))
    print("straight ", len(imagesDict["straight"]))
    print("right", len(imagesDict["right"]))
    returnArray = []
    returnArray.extend(imagesDict["right"])
    returnArray.extend(imagesDict["straight"])
    returnArray.extend(imagesDict["left"])

    return returnArray
            
    

In [12]:
lines = []
images = []
batch_size = 256
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

print(len(lines), " found")

for row in lines:
    steering_center = float(row[3])
    # read in images from center, left and right cameras
    img_center = ["IMG/" + row[0].split('\\')[-1], steering_center, "normal", "straight"]
    img_left = ["IMG/" + row[1].split('\\')[-1], steering_center, "normal", "left"]
    img_right = ["IMG/" + row[2].split('\\')[-1], steering_center, "normal", "right"]
    img_center_flipped = ["IMG/" + row[0].split('\\')[-1], steering_center*-1.0, "flipped", "straight"]
    #flipped, so right and left is exchanged
    img_left_flipped = ["IMG/" + row[1].split('\\')[-1], steering_center*-1.0, "flipped", "right"]
    img_right_flipped = ["IMG/" + row[2].split('\\')[-1], steering_center*-1.0, "flipped", "left"]

    # add images and angles to data set
    images.extend([img_center, img_left, img_right, img_center_flipped, img_left_flipped, img_right_flipped])
    
print(len(images), " imagepaths loaded")

images = GetEqualShareLeftRightStraight(images)



for image in images:
    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_center = float(image[1])

    steering_left = steering_center - correction
    steering_right = steering_center + correction
    if(image[3] == "left"):
        image[1] = image[1] + correction
    if(image[3] == "right"):
        image[1] = image[1] - correction
    #if(image[1] > 1.0 or image[1] < -1.0):
        #print(image)

for i in range(24):
    print(images[i])
        
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(images, test_size=0.1)
      
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size)
validation_generator = generator(validation_samples, batch_size)

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

model.add(Conv2D(24, (5, 5), strides=(2, 3), activation="relu"))
#model.add(MaxPooling2D())
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation="relu"))
model.add(Conv2D(48, (4, 4), strides=(2, 2), activation="relu"))
model.add(Conv2D(48, (3, 3), strides=(2, 2), activation="relu"))
model.add(Conv2D(64, (3, 3), strides=(1, 1), activation="relu"))
model.add(Flatten())
model.add(Dense(1000))
model.add(Dropout(0.4))
model.add(Dense(100))
model.add(Dropout(0.4))
model.add(Dense(50))
model.add(Dropout(0.4))
model.add(Dense(10))
model.add(Dense(1))

#adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer='adam')
model.summary()

model.fit_generator(train_generator, samples_per_epoch= len(train_samples)/batch_size, validation_data=validation_generator, nb_val_samples=len(validation_samples)/batch_size, epochs=5)
model.save('model.h5')

19738  found
118428  imagepaths loaded
left  28743
straight  60942
right 28743
left  57486
straight  60942
right 57486
['IMG/center_2017_10_27_15_51_44_434.jpg', 0.1030043, 'flipped', 'straight']
['IMG/left_2017_10_27_15_51_44_434.jpg', -0.09699570000000002, 'flipped', 'right']
['IMG/right_2017_10_27_15_51_44_434.jpg', 0.3030043, 'flipped', 'left']
['IMG/center_2017_10_27_15_51_44_502.jpg', 0.1416309, 'flipped', 'straight']
['IMG/left_2017_10_27_15_51_44_502.jpg', -0.05836910000000001, 'flipped', 'right']
['IMG/right_2017_10_27_15_51_44_502.jpg', 0.3416309, 'flipped', 'left']
['IMG/center_2017_10_27_15_51_44_569.jpg', 0.1502146, 'flipped', 'straight']
['IMG/left_2017_10_27_15_51_44_569.jpg', -0.04978540000000001, 'flipped', 'right']
['IMG/right_2017_10_27_15_51_44_569.jpg', 0.35021460000000004, 'flipped', 'left']
['IMG/center_2017_10_27_15_51_44_636.jpg', 0.1545064, 'flipped', 'straight']
['IMG/left_2017_10_27_15_51_44_636.jpg', -0.04549360000000002, 'flipped', 'right']
['IMG/right_201

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:84: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:84: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=618.445312..., epochs=5, validation_steps=68.71875, validation_data=<generator...)`


Epoch 1/5
619/618 [==============================] - 268s - loss: 0.0409 - val_loss: 0.0298
Epoch 2/5
619/618 [==============================] - 267s - loss: 0.0280 - val_loss: 0.0241
Epoch 3/5
619/618 [==============================] - 268s - loss: 0.0235 - val_loss: 0.0211
Epoch 4/5
619/618 [==============================] - 267s - loss: 0.0207 - val_loss: 0.0189
Epoch 5/5
619/618 [==============================] - 267s - loss: 0.0187 - val_loss: 0.0184


In [ ]:
#old
model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

model.add(Conv2D(16, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
#model.add(MaxPooling2D())
model.add(Conv2D(32, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
model.add(Conv2D(64, (5, 5), strides=(1, 2), activation="relu"))
model.add(Dropout(0.4))
model.add(Conv2D(96, (5, 5), strides=(2, 2), activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(128, (4, 4), strides=(1, 1), activation="relu"))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(520))
model.add(Dense(84))
model.add(Dense(1))


import csv
import cv2
import numpy as np
from keras import optimizers

lines = []
with open("driving_log.csv") as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
counter = 0
if(False):
    for line in lines:
        for i in range(3):
            sourcepath = line[i]
            filename = sourcepath.split('\\')[-1]
            currentpath = "IMG/" + filename
            image = cv2.imread(currentpath)
            images.append(image)
            measurements.append(float(line[3]))
            counter = counter + 1
        
for row in lines:
    steering_center = float(row[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    path = "..." # fill in the path to your training IMG directory
    img_center = cv2.imread("IMG/" + row[0].split('\\')[-1])
    img_left = cv2.imread("IMG/" + row[1].split('\\')[-1])
    img_right = cv2.imread("IMG/" + row[2].split('\\')[-1])

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)
    
x_train = np.array(images)
y_train = np.array(measurements)


from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x/255.0), input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

model.add(Conv2D(16, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
#model.add(MaxPooling2D())
model.add(Conv2D(32, (5, 5), strides=(2, 3), activation="relu"))
model.add(Dropout(0.6))
model.add(Conv2D(64, (5, 5), strides=(1, 2), activation="relu"))
model.add(Dropout(0.4))
model.add(Conv2D(96, (5, 5), strides=(2, 2), activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(128, (4, 4), strides=(1, 1), activation="relu"))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(520))
model.add(Dense(84))
model.add(Dense(1))

adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mse', optimizer=adam)
model.summary()

model.fit(x_train, y_train, validation_split=0.1, shuffle = True, epochs = 8)
model.save('model.h5')
